In [1]:
#import unicodedata
import lxml.html
import requests
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_columns', None)
import time
import os
import math
import json
import time
import datetime

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

now = datetime.datetime.now()

coreWOBA = ((2*(H+BB+HBP)) + TB)/PA  
wRC+ = 200 * coreWOBA - 100  
wOBA = .320 * coreWOBA

Explainer on how WAA links to WAR  
http://tangotiger.com/index.php/site/comments/how-does-wins-above-average-waa-link-to-wins-above-replacement-war?utm_source=dlvr.it&utm_medium=twitter  

OFF = batting and baserunning RAA  
DEF = defensive runs above average with positional adjustment  
OFF + DEF = RAA  
translation to WAR is complicated but is usually 9-10 runs per win  
game slice comes from the idea that position players get 4/7th, there are 38 PA in a typical game, so 4/7 divided by 38 is .015 game slices per PA  
should adjust the 38, esp for leagues that play a 7 inning game

In [2]:
def calc_pitching_war(p):
    GAMES   = p['GS'].sum() 
    GAMES = p['Outs'].sum()/3/8.5
    #print('GAMES:',GAMES)
    ER      = p['ER'].sum()
    #print('ER:',ER)
    INNINGS = p['Outs'].sum() / 3 
    #print('INNINGS:', INNINGS)
    IP_OUTS = p['Outs'].sum()
    #print('IP_OUTS:',IP_OUTS, ', really just total outs')

    GAME_SLICES = GAMES * 3 / 7 
    #print('GAME SLICES:', GAME_SLICES)
    INN_PER_GAME_SLICES    = INNINGS / GAME_SLICES
    #print('INN_PER_GAME_SLICES:', INN_PER_GAME_SLICES)
    GAME_SLICE_PER_IP_OUTS = GAME_SLICES / IP_OUTS
    #print('GAME_SLICES_PER_IP_OUTS:',GAME_SLICE_PER_IP_OUTS)
    ER_PER_IP_OUTS         = ER / IP_OUTS
    #print('ER_PER_IP_OUTS:', ER_PER_IP_OUTS)
    
    #Now we'll calculate ER better than average.  
    #Take your ER_per_IP_outs (0.158)  multiply by the number of IP_outs (or IP x 3) and subtract number of ER.
    #And we'll calculate Game Slices, or Individualized Games (iG): IP_outs x G_per_IP_outs (or 0.0163)
    #https://twitter.com/tangotiger/status/1445139885954936832?s=20
    p['ER_PM'] = (p['Outs'] * ER_PER_IP_OUTS) - p['ER']
    p['iG'] = (p['Outs'] * GAME_SLICE_PER_IP_OUTS)
    p['W+/-'] = p['ER_PM'] / 10

    #W+/- is ER+/- divided by 10 ... that's WAA (wins above average)
    #0.500 is simply iG/2
    #iW = iG/2 + WAA... that's Individualized Wins
    #iL = iG - iW... Individualized Losses
    #https://twitter.com/tangotiger/status/1445143538610626568?s=20
    p['0.500'] = p["iG"] / 2 
    p['iW']    = p['0.500'] + p['W+/-']
    p['iL']    = p["iG"] - p["iW"]

    #replacement level is .300 win% times iG (games)
    #And CoreWAR is iW minus replacement level
    #That's it... that's CoreWAR
    #There's 416 wins across the league.  That's the same thing as 4858 x (0.5 - 0.3) x 3/7.
    #Next post, we'll talk about all the adjustments you need
    #https://twitter.com/tangotiger/status/1445145998536450049?s=20

    p['repl_Level'] = p['iG'] * .3 
    p['CoreWAR_2'] = p['iW'] - p['repl_Level']
    return p

In [79]:
def add_woba(z):
    #(0.691×uBB + 0.722×HBP + 0.884×1B + 1.257×2B + 1.593×3B + 2.058×HR) / (AB + BB – IBB + SF + HBP)
    z['wOBA'] = round(((0.691*z['BB']) + (0.722*z['HBP']) + (0.884*z['1B']) + (1.257*z['2B']) + (1.593*z['3B']) + (2.058*z['HR'])) / (z['AB'] + z['BB'] + z['HBP'] + z['SF']),3)
    return z

def make_lg_avg(z):
    stats = ['GP', 'PA', 'AB', 'R', 'H', '1B', '2B', '3B', 'HR', 'RBI', 'BB', 'K', 'HBP', 'SB', 'CS', 'SF', 'SH', 'TB']
    #lgR = z.groupby(['Org', 'League', 'Year'])['R'].sum().reset_index(name='lgR')['lgR']
    #lgPA = z.groupby(['Org', 'League', 'Year'])['PA'].sum().reset_index(name='lgPA')['lgPA']
    z = z.groupby(['Org', 'League', 'Year']).agg({'GP':'mean', 'PA':'mean', 'AB':'mean', 'R':'mean', 'H':'mean', '1B':'mean', '2B':'mean', '3B':'mean', 'HR':'mean', 'RBI':'mean', 'BB':'mean', 'K':'mean', 'HBP':'mean', 'SB':'mean', 'CS':'mean', 'SF':'mean', 'SH':'mean', 'TB':'mean'}).reset_index()
    z['lgR'] = z.groupby(['Org', 'League', 'Year'])['R'].sum().reset_index(name='lgR')['lgR']
    z['lgPA'] = z.groupby(['Org', 'League', 'Year'])['PA'].sum().reset_index(name='lgPA')['lgPA']
    #z = z.merge(lgR, on=[['Org', 'League', 'Year']], how='inner')
    for i in stats:
        z[i] = round(z[i],1)
    add_rate_stats(z)
    add_lg_woba(z)
    z['wOBAscale'] = z['OBP']/z['wOBA']
    return z

def add_rate_stats(z):
    z['BA'] = round(z['H']/z['AB'],3)
    z['OBP'] = round((z['H']+z['BB']+z['HBP'])/(z['AB']+z['BB']+z['HBP']+z['SF']),3)
    z['SLG'] = round(z['TB']/z['AB'],3)
    z['OPS'] = round(z['SLG'] + z['OBP'],3)
    return z
def add_lg_woba(z):
    z['wOBA'] = round(((0.691*z['BB']) + (0.722*z['HBP']) + (0.884*z['1B']) + (1.257*z['2B']) + (1.593*z['3B']) + (2.058*z['HR'])) / (z['AB'] + z['BB'] + z['HBP'] + z['SF']),3)
    return z
def add_wRAA(z, avg):
    #wRAA formula = ((wOBA-lgwOBA)/wOBAScale)*PA;
    #z = z.merge(avg[['Org', 'League', 'Year', 'wOBA', 'wOBAscale']], on=['Org', 'League', 'Year'], suffixes=['', '_lg'], how='outer')
    z['wRAAc'] = round(((z['wOBA'] - z['wOBA_lg']) / z['wOBAscale'])*z['PA'],2)
    return z
def add_ops_plus(z, avg):
    #(OBP / lgOBP + SLG / lgSLG - 1) * 100
    z = z.merge(avg[['Org', 'League', 'Year', 'lgR', 'lgPA', 'BA', 'OBP', 'SLG', 'OPS', 'wOBA', 'wOBAscale']], on=['Org', 'League', 'Year'], suffixes=['', '_lg'], how='inner')
    z['OPS+'] = round((z['OBP']/z['OBP_lg'] + z['SLG']/z['SLG_lg'] -1)*100,0)
    return z

def add_wRC(z, avg):
    #wRC = (((wOBA-League wOBA)/wOBA Scale)+(League R/PA))*PA
    #z = z.merge(avg[['Org', 'League', 'Year', 'wOBA']], on=['Org', 'League', 'Year'], suffixes=['', '_lg'], how='outer')
    z['wRC'] = round((((z['wOBA'] - z['wOBA_lg']) / z['wOBAscale']) + (z['lgR'] / z['lgPA'])) * z['PA'],1)
    return z

def add_wRC_plus(z, avg):
    #wRC+ = (((wRAA/PA + League R/PA) + (League R/PA – Park Factor* League R/PA))/ (AL or NL wRC/PA excluding pitchers))*100
    z['wRC+'] = round(((z['wRAAc']/z['PA'] + z['lgR']/z['lgPA'])  / (z['lgR']/z['lgPA'])) * 100, 0)
    return z

def calc_hitting_war(df, org, lg, yr):
    mask = (df['Org']==org) & (df['League']==lg) & (df['Year']==yr)
    st_mask = (st['Org']==org) & (st['League']==lg) & (st['Year']==yr)
    num_teams = st[st_mask].Team.nunique()
    num_games = st[st_mask][['W', 'L', 'T']].sum().sum()
    
    wins_avail = num_games/2
    repl_wins = .297*wins_avail
    pos_wins_avail = (wins_avail - repl_wins) * (4/7)
    df.loc[mask, 'replacement_runs'] = pos_wins_avail * df[mask].R.sum() / wins_avail * df[mask]['PA'] / df[mask]['PA'].sum()
    df.loc[mask, 'replacement_runs'] = pos_wins_avail * df[mask].R.sum() / wins_avail * df[mask]['PA'] / df[mask].PA.sum()
    df.loc[mask, 'WAR'] = (df[mask]['wRAAc']+df[mask]['replacement_runs']) / (df[mask].R.sum() / wins_avail)
    return

In [92]:
org='MSCR'
lg='Caspian'
yr=2009
mask = (df['Org']==org) & (df['League']==lg) & (df['Year']==yr)
st_mask = (st['Org']==org) & (st['League']==lg) & (st['Year']==yr)

In [93]:
df[mask]

,PID,First,Last,Org,League,Team,Year,GP,GS,PA,AB,R,H,1B,2B,3B,HR,RBI,BB,K,HBP,SB,CS,SH,SF,TB,BA,OBP,SLG,OPS,opsPlus,wRAA,wOBA,wRC,wRCPlus,All-Star,DP,ROE,FC,LOB,Outs,lgR,lgPA,BA_lg,OBP_lg,SLG_lg,OPS_lg,wOBA_lg,wOBAscale,OPS+,wRAAc,wRC+,replacement_runs,WAR
441,149,Aziel,Bell,MSCR,Caspian,Annabelle Lee,2009,1,0,4,4,1,2.0,1.0,1.0,0,0,0,0,0.0,0,0,0,0.0,0.0,3,0.500,0.500,0.750,1.250,NaN,NaN,0.535,1.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.615385,23.307692,0.38,0.494,0.545,1.039,0.453,1.090508,139.0,0.30,123,0.525013,0.125002
442,484,Andy,DeRocher,MSCR,Caspian,Annabelle Lee,2009,10,0,34,28,7,11.0,10.0,1.0,0,0,6,5,1.0,1,7,0,0.0,0.0,12,0.393,0.500,0.429,0.929,NaN,NaN,0.420,10.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.615385,23.307692,0.38,0.494,0.545,1.039,0.453,1.090508,80.0,-1.03,91,4.462608,0.520092
443,865,James,Hanson,MSCR,Caspian,Annabelle Lee,2009,9,0,25,16,2,0.0,0.0,0.0,0,0,0,9,11.0,0,4,0,0.0,0.0,0,0.000,0.360,0.000,0.360,NaN,NaN,0.249,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.615385,23.307692,0.38,0.494,0.545,1.039,0.453,1.090508,-27.0,-4.68,43,3.281330,-0.211920
444,1145,Mike,Kelly,MSCR,Caspian,Annabelle Lee,2009,1,0,4,2,0,0.0,0.0,0.0,0,0,2,0,0.0,0,0,0,0.0,2.0,0,0.000,0.000,0.000,0.000,NaN,NaN,0.000,-0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.615385,23.307692,0.38,0.494,0.545,1.039,0.453,1.090508,-100.0,-1.66,-27,0.525013,-0.171968
445,1279,Konley,Kuehnl,MSCR,Caspian,Annabelle Lee,2009,9,0,27,22,5,5.0,3.0,2.0,0,0,3,5,12.0,0,2,0,0.0,0.0,7,0.227,0.370,0.318,0.688,NaN,NaN,0.319,5.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.615385,23.307692,0.38,0.494,0.545,1.039,0.453,1.090508,33.0,-3.32,62,3.543836,0.033915
446,1310,Ralph,Lansing,MSCR,Caspian,Annabelle Lee,2009,8,0,21,18,5,2.0,2.0,0.0,0,0,2,3,9.0,0,1,0,0.0,0.0,2,0.111,0.238,0.111,0.349,NaN,NaN,0.183,1.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.615385,23.307692,0.38,0.494,0.545,1.039,0.453,1.090508,-31.0,-5.20,24,2.756317,-0.370255
447,1431,Pete,Mand,MSCR,Caspian,Annabelle Lee,2009,9,0,36,29,12,13.0,7.0,3.0,2,1,16,5,4.0,1,5,0,0.0,1.0,23,0.448,0.528,0.793,1.321,NaN,NaN,0.538,14.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.615385,23.307692,0.38,0.494,0.545,1.039,0.453,1.090508,152.0,2.81,124,4.725115,1.141684
448,1574,Tim,Moon,MSCR,Caspian,Annabelle Lee,2009,7,0,23,21,5,8.0,5.0,2.0,0,0,4,1,7.0,0,9,0,0.0,1.0,9,0.381,0.391,0.429,0.820,NaN,NaN,0.332,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.615385,23.307692,0.38,0.494,0.545,1.039,0.453,1.090508,58.0,-2.55,66,3.018823,0.071034
449,1682,Chris,Nye,MSCR,Caspian,Annabelle Lee,2009,8,0,26,20,7,8.0,8.0,0.0,0,0,3,6,3.0,0,8,0,0.0,0.0,8,0.400,0.538,0.400,0.938,NaN,NaN,0.431,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.615385,23.307692,0.38,0.494,0.545,1.039,0.453,1.090508,82.0,-0.52,94,3.412583,0.438270
450,1781,Brent,Perzentka,MSCR,Caspian,Annabelle Lee,2009,9,0,35,25,20,11.0,7.0,3.0,1,0,5,6,4.0,3,15,0,1.0,0.0,16,0.440,0.588,0.640,1.228,NaN,NaN,0.525,13.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.615385,23.307692,0.38,0.494,0.545,1.039,0.453,1.090508,136.0,2.31,120,4.593861,1.046040


In [88]:
calc_hitting_war(df, org, lg, yr)

In [90]:
df = pd.read_csv('C:\\Users\\pddnh\\Documents\\GitHub\\baseball\\Master_Hitting.csv', encoding='cp1252')
df = add_rate_stats(df)
df['OBP'].fillna(0, inplace=True)
df['SLG'].fillna(0, inplace=True)
df['OPS'].fillna(0, inplace=True)
df = add_woba(df)
h_lg_avg = make_lg_avg(df)
df = add_ops_plus(df, h_lg_avg)
df = add_wRAA(df, h_lg_avg)
df = add_wRC(df, h_lg_avg)
df = add_wRC_plus(df, h_lg_avg)
df['wRC+'].fillna(0, inplace=True)
df['wRC+'] = df['wRC+'].astype(int)

p = pd.read_csv('C:\\Users\\pddnh\\Documents\\GitHub\\baseball\\Master_Pitching.csv')
st = pd.read_csv('C:\\Users\\pddnh\\Documents\\GitHub\\baseball\\Standings.csv')
oly = df[['Org', 'League', 'Year']].sort_values('Year').drop_duplicates().reset_index()

In [17]:
oly

,index,Org,League,Year
0,0,MSCR,Pacific,2003
1,22,MSCR,Pacific,2004
2,37,MSCR,Pacific,2005
3,59,MSCR,Pacific,2006
4,55,MSCR,Northern,2006
...,...,...,...,...
76,8084,RRL,Southern,2021
77,7744,MABL,45+,2021
78,7537,MABL,18+,2021
79,7593,MABL,35+,2021


In [91]:
for i, row in oly.iterrows():
    print(i, row['Org'], row['League'], row['Year'])
    calc_hitting_war(df, row['Org'], row['League'], row['Year'])

0 MSCR Pacific 2003
1 MSCR Pacific 2004
2 MSCR Pacific 2005
3 MSCR Pacific 2006
4 MSCR Northern 2006
5 MSCR Pacific 2007
6 MABL 18+ 2007
7 RRL Southern 2008
8 MSCR Pacific 2008
9 RRL Northern 2008
10 MSCR Northern 2008
11 MABL 18+ 2008
12 RRL Southern 2009
13 MSCR Pacific 2009
14 RRL Northern 2009
15 MSCR Caspian 2009


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in true_divide
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in true_divide
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: invalid value en

16 MABL 18+ 2009
17 RRL Southern 2010
18 MSCR Pacific 2010
19 RRL Northern 2010
20 MSCR Caspian 2010
21 MABL 18+ 2010
22 RRL Southern 2011
23 MSCR Pacific 2011
24 RRL Northern 2011
25 MSCR Caspian 2011
26 MABL 18+ 2011
27 RRL Southern 2012
28 RRL Northern 2012
29 MSCR Pacific 2012
30 MABL 18+ 2012
31 MABL 35+ 2012


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in true_divide


32 MSCR Caspian 2012
33 RRL Southern 2013
34 MSCR Pacific 2013
35 RRL Northern 2013
36 MSCR Northern 2013
37 MABL 18+ 2013
38 RRL Southern 2014
39 RRL Northern 2014
40 MSCR Pacific 2014
41 MSCR Northern 2014
42 MABL 18+ 2014
43 RRL Southern 2015
44 MSCR Pacific 2015
45 RRL Northern 2015
46 MSCR Northern 2015
47 MABL 18+ 2015
48 RRL Northern 2016
49 MSCR Pacific 2016
50 RRL Southern 2016
51 MABL 45+ 2016
52 MABL 35+ 2016
53 MABL 18+ 2016
54 RRL Northern 2017
55 RRL Southern 2017
56 MABL 18+ 2017
57 MABL 35+ 2017
58 MABL 17+ 2017
59 MABL 55+ 2017
60 MABL 45+ 2017
61 RRL Northern 2018
62 RRL Southern 2018
63 MABL 55+ 2018
64 MABL 18+ 2018
65 MABL 17+ 2018
66 MABL 45+ 2018
67 MABL 35+ 2018
68 RRL Northern 2019
69 RRL Southern 2019
70 MABL 35+ 2019
71 MABL 18+ 2019
72 RRL Southern 2020
73 RRL Northern 2020
74 RRL Northern 2021
75 MABL 55+ 2021
76 RRL Southern 2021
77 MABL 45+ 2021
78 MABL 18+ 2021
79 MABL 35+ 2021


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in true_divide
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in true_divide


80 MABL 17+ 2021


In [94]:
pp = pd.DataFrame()
for i, row in oly.iterrows():
    pp = pp.append(calc_pitching_war(p[(p['Org']==row['Org']) & (p['League']==row['League']) & (p['Year']==row['Year'])]))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [19]:
df[(df['League']=='35+') & (df['Year']==2018)].sort_values('WAR')

,PID,First,Last,Org,League,Team,Year,GP,GS,PA,AB,R,H,1B,2B,3B,HR,RBI,BB,K,HBP,SB,CS,SH,SF,TB,BA,OBP,SLG,OPS,opsPlus,wRAA,wOBA,wRC,wRCPlus,All-Star,DP,ROE,FC,LOB,Outs,lgR,lgPA,BA_lg,OBP_lg,SLG_lg,OPS_lg,wOBA_lg,wOBAscale,OPS+,wRAAc,wRC+,replacement_runs,WAR
5882,1354,Joshua,Lemberger,MABL,35+,Madison Braves,2018,11,11,26,25,3,2.0,2.0,0.0,0,0,0,1,12.0,0,1,1,0.0,0.0,2,0.080,0.115,0.080,0.195,0.0,NaN,0.095,-0.9,NaN,NaN,0.0,0.0,1.0,0.0,NaN,6.977612,31.910448,0.333,0.432,0.423,0.855,0.382,1.13089,-54.0,-6.60,-16,2.283834,-0.258508
5866,1070,Jeremy,Johanski,MABL,35+,HomeBrewers,2018,12,11,38,34,9,4.0,4.0,0.0,0,0,2,4,8.0,0,1,0,0.0,0.0,4,0.118,0.211,0.118,0.329,0.0,NaN,0.166,1.1,NaN,NaN,0.0,5.0,1.0,0.0,NaN,6.977612,31.910448,0.333,0.432,0.423,0.855,0.382,1.13089,-23.0,-7.26,13,3.337911,-0.234906
5945,494,Heith,Dey,MABL,35+,Mules,2018,15,10,37,30,5,3.0,3.0,0.0,0,0,0,7,8.0,0,0,0,0.0,0.0,3,0.100,0.270,0.100,0.370,0.0,NaN,0.202,2.2,NaN,NaN,0.0,3.0,1.0,0.0,NaN,6.977612,31.910448,0.333,0.432,0.423,0.855,0.382,1.13089,-14.0,-5.89,27,3.250071,-0.158113
5948,867,Todd,Hardin,MABL,35+,Mules,2018,12,8,27,21,1,1.0,1.0,0.0,0,0,0,3,13.0,3,0,0,0.0,0.0,1,0.048,0.259,0.048,0.307,0.0,NaN,0.190,1.3,NaN,NaN,0.0,1.0,0.0,0.0,NaN,6.977612,31.910448,0.333,0.432,0.423,0.855,0.382,1.13089,-29.0,-4.58,22,2.371674,-0.132263
5896,489,Jeremy,Deuchars,MABL,35+,Mammoths,2018,13,13,44,36,9,5.0,5.0,0.0,0,0,2,5,11.0,3,1,0,0.0,0.0,5,0.139,0.295,0.139,0.434,0.0,NaN,0.228,3.6,NaN,NaN,0.0,1.0,3.0,0.0,NaN,6.977612,31.910448,0.333,0.432,0.423,0.855,0.382,1.13089,1.0,-5.99,38,3.864950,-0.127276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5905,2193,Dan,Sherman,MABL,35+,Mammoths,2018,11,11,41,29,15,17.0,14.0,3.0,0,0,3,10,2.0,2,8,2,0.0,0.0,20,0.586,0.707,0.690,1.397,0.0,NaN,0.598,16.8,NaN,NaN,0.0,3.0,2.0,0.0,NaN,6.977612,31.910448,0.333,0.432,0.423,0.855,0.382,1.13089,227.0,7.83,187,3.601431,0.684663
5846,876,Dan,Haueter,MABL,35+,Athletics,2018,15,15,58,51,15,25.0,17.0,5.0,0,3,25,7,2.0,0,4,1,0.0,0.0,39,0.490,0.552,0.765,1.317,0.0,NaN,0.557,21.7,NaN,NaN,0.0,2.0,0.0,0.0,NaN,6.977612,31.910448,0.333,0.432,0.423,0.855,0.382,1.13089,209.0,8.98,171,5.094707,0.842977
5887,1770,Steve,Pennekamp,MABL,35+,Madison Braves,2018,12,10,42,33,7,21.0,16.0,3.0,0,2,7,9,1.0,0,8,0,0.0,0.0,30,0.636,0.714,0.909,1.623,0.0,NaN,0.673,20.0,NaN,NaN,0.0,1.0,0.0,0.0,NaN,6.977612,31.910448,0.333,0.432,0.423,0.855,0.382,1.13089,280.0,10.81,218,3.689270,0.868405
5926,304,Joe,Buss,MABL,35+,Monarchs,2018,16,16,67,55,21,30.0,23.0,7.0,0,0,13,10,1.0,1,11,0,0.0,1.0,37,0.545,0.612,0.673,1.285,0.0,NaN,0.549,24.5,NaN,NaN,0.0,4.0,1.0,0.0,NaN,6.977612,31.910448,0.333,0.432,0.423,0.855,0.382,1.13089,201.0,9.89,168,5.885265,0.944829
